# Data Cleaning

In deze notebook gaan we werken met een dataset die de gebeurtenissen tijdens een reeks wedstrijden over een periode van 4 jaar.
De link naar deze dataset is: https://www.kaggle.com/secareanualin/football-events.
Met deze kan je reeds heel wat informatie vinden over wat er in deze dataset zit. 
In de dataset zitten er drie files:
* events.csv Bevat data over de gebeurtenissen van elke wedstrijd.
* ginf.csv - Bevat metadata en win-kansen van elke wedstrijd.
* dictionary.txt Bevat een tekstuele weergave van de verschillende categorieen

Hieronder staan alle imports die gebruikt worden in de loop van de notebook.

In [1]:
# loading of the dataset
import opendatasets as od
from zipfile import ZipFile
import pandas as pd
import numpy as np

## Inladen dataset

De eerste stap is om de dataset te downloaden vanuit kaggle met de opendatasets library.
Aangezien de dataset meerdere bestanden bevat kan het zijn dat het wordt gedownload als zip (controleer dit). 
Unzip daarom eerst het bestand indien nodig.
Laad daarna de twee csv bestanden in in een dataframe.
De code voor deze stappen is:

In [ ]:
# download
od.download("https://www.kaggle.com/secareanualin/football-events")

In [2]:
# unzip if necessary en load data in
df_events = pd.read_csv(".\\football-events\\events.csv")
df_ginf = pd.read_csv(".\\football-events\\ginf.csv")

## Exploratory Data Analysis

De read_csv functie heeft een dataframe aangemaakt en alle data van de csv ingeladen in dit object.
Pandas voorziet daarnaast een aantal eenvoudige methodes om heel wat informatie uit de dataset op te halen.
Een overzicht van alle functies die mogelijk zijn met dataframes vind je hier: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html.
We kunnen bijvoorbeeld eenvoudig het aantal rijen gaan opvragen maar ook een aantal rijen gaan bekijken. 
Dit kan bijvoorbeeld met de volgende code:

In [3]:
df_events.info()
df_events.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 941009 entries, 0 to 941008
Data columns (total 22 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id_odsp        941009 non-null  object 
 1   id_event       941009 non-null  object 
 2   sort_order     941009 non-null  int64  
 3   time           941009 non-null  int64  
 4   text           941009 non-null  object 
 5   event_type     941009 non-null  int64  
 6   event_type2    214293 non-null  float64
 7   side           941009 non-null  int64  
 8   event_team     941009 non-null  object 
 9   opponent       941009 non-null  object 
 10  player         880009 non-null  object 
 11  player2        291310 non-null  object 
 12  player_in      51715 non-null   object 
 13  player_out     51738 non-null   object 
 14  shot_place     227459 non-null  float64
 15  shot_outcome   228498 non-null  float64
 16  is_goal        941009 non-null  int64  
 17  location       467067 non-nul

id_odsp               0
id_event              0
sort_order            0
time                  0
text                  0
event_type            0
event_type2      726716
side                  0
event_team            0
opponent              0
player            61000
player2          649699
player_in        889294
player_out       889271
shot_place       713550
shot_outcome     712511
is_goal               0
location         473942
bodypart         711824
assist_method         0
situation        711872
fast_break            0
dtype: int64

Probeer nu zelf de volgende zaken op te zoeken:
* Haal willekeurig 20 rijen uit de dataset
* Alle spelers die betrokken zijn geweest bij een event (geen duplicaten)
* Hoeveel goalen zijn er gemaakt in de geregistreerde events. Haal het correcte event type uit de dictionary.txt
* Bestudeer de inhoud van het ginf dataset, besteed vooral aandacht aan de ontbrekende data
* Voeg aan het ginf dataframe twee kolommen toe, 1 voor het aantal events van de thuisploeg en 1 voor het aantal events van de andere ploeg.

In [31]:
# punt 1
df_events.sample(20)

#punt 2
#group by om te groeperen op de kolom player (alle identieke waarden worden gecombineerd)
#size is een aggregatiemethode nodig om het aantal keer te berekenen dat elke speler voorkomt in een event
df_events.groupby("player").size().sort_values()

#punt 3
df_events.is_goal.sum()

#punt 4
events_per_match = df_events.groupby(["id_odsp", "side"]).size().reset_index()
events_per_match_home = events_per_match[events_per_match.side == 1].set_index("id_odsp")
events_per_match_away = events_per_match[events_per_match.side == 2].set_index("id_odsp")
df_ginf["events_home"] = events_per_match_home[0]

df_ginf

,id_odsp,link_odsp,adv_stats,date,league,season,country,ht,at,fthg,ftag,odd_h,odd_d,odd_a,odd_over,odd_under,odd_bts,odd_bts_n,events_home
0,UFot0hit/,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN,NaN
1,Aw5DflLH/,/soccer/germany/bundesliga-2011-2012/augsburg-...,True,2011-08-06,D1,2012,germany,FC Augsburg,SC Freiburg,2,2,2.36,3.60,3.40,NaN,NaN,NaN,NaN,NaN
2,bkjpaC6n/,/soccer/germany/bundesliga-2011-2012/werder-br...,True,2011-08-06,D1,2012,germany,Werder Bremen,Kaiserslautern,2,0,1.83,4.20,4.80,NaN,NaN,NaN,NaN,NaN
3,CzPV312a/,/soccer/france/ligue-1-2011-2012/paris-sg-lori...,True,2011-08-06,F1,2012,france,Paris Saint-Germain,Lorient,0,1,1.55,4.50,9.40,NaN,NaN,NaN,NaN,NaN
4,GUOdmtII/,/soccer/france/ligue-1-2011-2012/caen-valencie...,True,2011-08-06,F1,2012,france,Caen,Valenciennes,1,0,2.50,3.40,3.45,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10107,xAkY8l6R/,/soccer/italy/serie-a/genoa-crotone-xAkY8l6R/,True,2017-01-22,I1,2017,italy,Genoa,Crotone,2,2,1.97,4.35,8.00,1.95,2.03,2.03,1.86,NaN
10108,xSU9scI9/,/soccer/england/premier-league/chelsea-hull-ci...,True,2017-01-22,E0,2017,england,Chelsea,Hull,2,0,1.19,8.50,20.00,1.54,2.68,2.40,1.66,NaN
10109,xY7uZwOI/,/soccer/france/ligue-1/monaco-lorient-xY7uZwOI/,True,2017-01-22,F1,2017,france,AS Monaco,Lorient,4,0,1.32,6.24,11.50,1.53,3.08,1.80,2.25,NaN
10110,YyeGxMX8/,/soccer/spain/laliga/betis-gijon-YyeGxMX8/,True,2017-01-22,SP1,2017,spain,Real Betis,Sporting Gijon,0,0,1.74,4.07,5.90,2.20,1.89,2.05,1.86,NaN


## Data Cleaning

### Ontbrekende data

In eerste instantie zijn er de volgende mogelijkheden om om te gaan met ontbrekende data (Null or Nan values)
* Negeren
* Verwijder rijen met null waarden
* Verwijder kolommen met null waarden
* Kies een waarde om de NaN waarden in te vullen
* Gebruik duplicaten om de waarde in te vullen

Je kan gebruik maken van de volgende functies:
* isna or notna om aan te geven welke elementen ontbrekende of ingevulde data bevatten 
* dropna drop columns or rows containing missing values  (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)
* fillna fill in missing values https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html#pandas.DataFrame.fillna

Deze functies hebben de volgende resultaten

In [35]:
# volledige kopie maken zodat het niet steeds vanaf de disk ingeladen worden
df_events_copy = df_events.copy()

display(df_events_copy.shape)
display(df_events_copy.info())

# standaard delete rows met minstens 1 nan waarde
df_events_copy.dropna()
# met axis=1 verwijder kolommen
df_events_copy.dropna(axis=1)

df_events_copy.fillna(-1)

(941009, 22)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 941009 entries, 0 to 941008
Data columns (total 22 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id_odsp        941009 non-null  object 
 1   id_event       941009 non-null  object 
 2   sort_order     941009 non-null  int64  
 3   time           941009 non-null  int64  
 4   text           941009 non-null  object 
 5   event_type     941009 non-null  int64  
 6   event_type2    214293 non-null  float64
 7   side           941009 non-null  int64  
 8   event_team     941009 non-null  object 
 9   opponent       941009 non-null  object 
 10  player         880009 non-null  object 
 11  player2        291310 non-null  object 
 12  player_in      51715 non-null   object 
 13  player_out     51738 non-null   object 
 14  shot_place     227459 non-null  float64
 15  shot_outcome   228498 non-null  float64
 16  is_goal        941009 non-null  int64  
 17  location       467067 non-nul

None

,id_odsp,id_event,sort_order,time,text,event_type,event_type2,side,event_team,opponent,...,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break
0,UFot0hit/,UFot0hit1,1,2,Attempt missed. Mladen Petric (Hamburg) left f...,1,12.0,2,Hamburg SV,Borussia Dortmund,...,-1,-1,6.0,2.0,0,9.0,2.0,1,1.0,0
1,UFot0hit/,UFot0hit2,2,4,"Corner, Borussia Dortmund. Conceded by Dennis...",2,-1.0,1,Borussia Dortmund,Hamburg SV,...,-1,-1,-1.0,-1.0,0,-1.0,-1.0,0,-1.0,0
2,UFot0hit/,UFot0hit3,3,4,"Corner, Borussia Dortmund. Conceded by Heiko ...",2,-1.0,1,Borussia Dortmund,Hamburg SV,...,-1,-1,-1.0,-1.0,0,-1.0,-1.0,0,-1.0,0
3,UFot0hit/,UFot0hit4,4,7,Foul by Sven Bender (Borussia Dortmund).,3,-1.0,1,Borussia Dortmund,Hamburg SV,...,-1,-1,-1.0,-1.0,0,-1.0,-1.0,0,-1.0,0
4,UFot0hit/,UFot0hit5,5,7,Gokhan Tore (Hamburg) wins a free kick in the ...,8,-1.0,2,Hamburg SV,Borussia Dortmund,...,-1,-1,-1.0,-1.0,0,2.0,-1.0,0,-1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941004,z5L2OT5E/,z5L2OT5E123,123,92,Lucas Torreira (Sampdoria) wins a free kick in...,8,-1.0,2,Sampdoria,Atalanta,...,-1,-1,-1.0,-1.0,0,2.0,-1.0,0,-1.0,0
941005,z5L2OT5E/,z5L2OT5E124,124,93,"Corner, Sampdoria. Conceded by Andrea Masiello.",2,-1.0,2,Sampdoria,Atalanta,...,-1,-1,-1.0,-1.0,0,-1.0,-1.0,0,-1.0,0
941006,z5L2OT5E/,z5L2OT5E125,125,93,Attempt missed. Fabio Quagliarella (Sampdoria)...,1,12.0,2,Sampdoria,Atalanta,...,-1,-1,8.0,2.0,0,9.0,1.0,1,3.0,0
941007,z5L2OT5E/,z5L2OT5E126,126,94,Alberto Grassi (Atalanta) wins a free kick on ...,8,-1.0,1,Atalanta,Sampdoria,...,-1,-1,-1.0,-1.0,0,4.0,-1.0,0,-1.0,0


We zien dat de manier waarop we dropna uitvoeren een grote impact kan hebben op het resultaat.
In de eerste situatie bekomen we een lege dataset. Dit komt omdat er op elke rij minstens 1 kolom is die een Nan value bevat.

In het tweede voorbeeld, worden alle kolommen verwijderd die minstens 1 Nan value bevat. Op deze manier hebben we nog evenveel elementen maar het aantal kolommen is sterk gereduceerd. Door dit zonder controle te gebruiken kan veel nuttige informatie verwijderd worden. Merk op dat de 4 kolommen met de speler informatie verwijderd zijn. En ook de kolommen die meer informatie bevatten over het type event.

Het merendeel van de verwijderde kolommen bevatten dus nuttige informatie en willen we daarom niet verwijderen.
Om deze reden hebben we een meer manuele aanpak nodig.
De kolommen met informatie over het event (bvb shot_place) zijn nodig en mogen dus blijven staan. 
Nan geeft dan enkel weer dat de waarde niet van toepassing is.
De 4 kolommen met speler informatie kan echter gereduceerd worden tot twee kolommen (1 voor elk team) waarbij player2 niet altijd ingevuld moet zijn.
Dit kan bewezen worden als volgt: 

In [49]:
nans = df_events_copy[["player", "player_in"]].isna()
print(sum(nans["player"]))
print(sum(nans["player_in"]) + sum(nans["player"]))

# welke waarden zijn dubbel nan?
print(sum(nans["player_in"] & nans["player"]))
df_events_copy.loc[941005]
print(sum(~nans["player_in"] & ~nans["player"]))
# andere operatoren op series van booleans
print(sum(nans["player_in"] | nans["player"])) #of
print(sum(nans["player_in"] ^ nans["player"])) #xor

# verwijder dubbel nan waarden
df_events_copy_reduced = df_events_copy[nans["player"] ^ nans["player_in"]]
df_events_copy_reduced.shape

# merge player en player_in kolom
def merge_player(row):
    display(row)
    if row.isnull().player:
        return row.player_in
    else:
        return row.player
    
# dit is niet de snelste manier maar overloop het volledige dataframe 1 voor 1
#df_events_copy_reduced.apply(merge_player, axis=1)

nans_player = df_events_copy_reduced.isnull().player
# ga op zoek naar een index waar player = Nan    (197869)
#nans_player.sort_values(ascending=False)
df_events_copy_reduced["player"]
df_events_copy_reduced.loc[nans_player, "player"] = df_events_copy_reduced.loc[nans_player, "player_in"]
df_events_copy_reduced.loc[197869, ["player", "player_in"]]

# isnull() isna()
# .loc
# = 

# zelfde voor player2
nans_player2 = df_events_copy_reduced.isnull().player2
# ga op zoek naar een index waar player = Nan    (571327)
display(nans_player2.sort_values(ascending=False))
df_events_copy_reduced.loc[nans_player2, "player2"] = df_events_copy_reduced.loc[nans_player2, "player_out"]
df_events_copy_reduced.loc[571327, ["player2", "player_out"]]

61000
950294
9285
0
941009
931724


467281     True
571337     True
571327     True
571328     True
571329     True
          ...  
700491    False
700490    False
167055    False
167056    False
0         False
Name: player2, Length: 931724, dtype: bool

player2       NaN
player_out    NaN
Name: 571327, dtype: object

Met bovenstaande resultaten zien we dat in bijna alle gevallen player ofwel player_in ingevuld is maar nooit allebei.
Dit houdt in dat we deze twee kolommen eenvoudig kunnen samenvoegen. 
Hetzelfde geldt voor de kolommen player2 en player_out.
Nog een belangrijke opmerking is dat er een aantal rijen zijn waar er geen informatie over spelers aanwezig is.
Deze rijen kunnen verwijderd worden.
Ook kan de kolom "text" met de tekstuele uitleg van het event verwijderd worden.

Dit doen we hier:

In [54]:
df_events_copy_reduced.drop("text", axis=1)

KeyError: "['text'] not found in axis"

Oefening: Zijn er na het mergen van deze twee kolommen nog gevallen waar player null is maar player2 niet? In het geval dat er nog zo rijen zijn, verplaats de waarde van de player2 kolom naar de player kolom (enkel voor deze rijen). Nadat dit gedaan is hebben we zoveel mogelijk gedaan en kunnen we de overige rijen waarvoor de player feature null is laten vallen. 

### One hot encoding

In deze dataset wordt er vaak een enum gebruikt om een categorie aan te duiden. 
Dit is een compacte manier om een enum in een dataset op te slaan maar dit geeft vaak problemen bij het opbouwen van modellen.
Dit gaan we in meer detail bekijken bij classificatie/clustering.

Om dit op te lossen kunnen gebruik maken van one-hot encoding.
In pandas kan dat door gebruik te maken van de functie pd.get_dummies()

In [68]:
# categorieke data (klassen) om te zetten naar numerieke waarden
# ordinal encoding klasse 1 tot N in 1 kolom
# one hot encoding 1 klasse per kolom met true/false waarden
df_events_copy_reduced
# bodypart omzetten naar one-hot encoding
# pandas
one_hot = pd.get_dummies(df_events_copy_reduced.bodypart, prefix="part", dtype=bool)

# sklearn
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
encoded = enc.fit_transform(df_events_copy_reduced.bodypart.values.reshape(-1, 1))

# test ordinal encoding - sklearn
df_events_copy_reduced.event_team
from sklearn.preprocessing import OrdinalEncoder # werkt beter met meerdere kolommen
enc = OrdinalEncoder()
enc.fit_transform(df_events_copy_reduced.event_team.values.reshape(-1,1))

# of met label encoder (werkt beter als je werkt met 1 kolom) 
# vooral bedoeld voor labels/targets
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
enc.fit_transform(df_events_copy_reduced.event_team)

array([ 58,  20,  20, ..., 110,  11, 110])

### Data Format Correcting

Buiten het verwijderen of toevoegen van kolommen kan het ook belangrijk zijn om de waarden van individuele elementen te corrigeren/aan te passen. 
Dit kan een heel aantal zaken omvatten zoals het corrigeren van typo's, omzetten van datatypes, etc.
Laten we eerst eens de One-Hot Encoded values omzetten van 0/1 naar true/false.

In [74]:
# changing datatype of column or df
one_hot["part_1.0"] = one_hot["part_1.0"].astype("int")
one_hot

,part_1.0,part_2.0,part_3.0
0,0,True,False
1,0,False,False
2,0,False,False
3,0,False,False
4,0,False,False
...,...,...,...
941003,0,False,False
941004,0,False,False
941006,1,False,False
941007,0,False,False


Daarnaast moeten er ook vaak woorden in tekstuele velden gecorrigeerd of vertaald worden. 
Om hiervan een voorbeeld te geven ga ik in de player en player2 kolommen de naam van "Romelu Lukaku" hernoemen naar "Big Rom".
De volgende code voert dit uit door gebruik te maken van de [replace](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html) functie:

In [77]:
df_events_copy_reduced.replace(to_replace="Hamburg SV", value="Bicky")

,id_odsp,id_event,sort_order,time,event_type,event_type2,side,event_team,opponent,player,...,player_in,player_out,shot_place,shot_outcome,is_goal,location,bodypart,assist_method,situation,fast_break
0,UFot0hit/,UFot0hit1,1,2,1,12.0,2,Bicky,Borussia Dortmund,mladen petric,...,NaN,NaN,6.0,2.0,0,9.0,2.0,1,1.0,0
1,UFot0hit/,UFot0hit2,2,4,2,NaN,1,Borussia Dortmund,Bicky,dennis diekmeier,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
2,UFot0hit/,UFot0hit3,3,4,2,NaN,1,Borussia Dortmund,Bicky,heiko westermann,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
3,UFot0hit/,UFot0hit4,4,7,3,NaN,1,Borussia Dortmund,Bicky,sven bender,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
4,UFot0hit/,UFot0hit5,5,7,8,NaN,2,Bicky,Borussia Dortmund,gokhan tore,...,NaN,NaN,NaN,NaN,0,2.0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941003,z5L2OT5E/,z5L2OT5E122,122,92,3,NaN,1,Atalanta,Sampdoria,leonardo spinazzola,...,NaN,NaN,NaN,NaN,0,NaN,NaN,0,NaN,0
941004,z5L2OT5E/,z5L2OT5E123,123,92,8,NaN,2,Sampdoria,Atalanta,lucas torreira,...,NaN,NaN,NaN,NaN,0,2.0,NaN,0,NaN,0
941006,z5L2OT5E/,z5L2OT5E125,125,93,1,12.0,2,Sampdoria,Atalanta,fabio quagliarella,...,NaN,NaN,8.0,2.0,0,9.0,1.0,1,3.0,0
941007,z5L2OT5E/,z5L2OT5E126,126,94,8,NaN,1,Atalanta,Sampdoria,alberto grassi,...,NaN,NaN,NaN,NaN,0,4.0,NaN,0,NaN,0


Een andere grote bron van data formaten dat aangepast moet worden komt voort uit kolommen die datums bevatten. Deze worden standaard ingelezen als een tekst veldje dus moet je deze zelf gaan omzetten. Dit kan je zien in de ginf dataset waar de "date" kolom van het type object is. Dit is in numpy de indicatie dat het een tekstveld is.

In de dataframe zien we dat de datums erin staan als "%Y-%m-%d" (jaar als 4 cijfers, rest als twee). 
Voor de notatie van andere dataformaten vind je [hier](http://strftime.org/) meer informatie.
Met behulp van de [to_datetime](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html) functie kan je deze kolom omzetten naar datums.
Zoek na deze omzetting eens het datatype op van deze kolom.
Je ziet dat het nu geen object meer is maar een correcte datum.

Merk op dat deze manier van aanpakken enkel werkt wanneer de hele tabel hetzelfde dataformaat heeft.
Indien er meerdere dataformaten gebruikt zijn (bijvoorbeeld door manuele input), dan kan je het nog steeds doen maar dan moet panda een inschatting maken van het gewenste formaat. Dit kan door het argument "infer_datetime_format=True".
Hoewel dit mogelijk is zijn er twee redenen waarom we dit niet altijd gebruiken, namelijk
* Het duurt veel langer om de tabel om te zetten.
* Het gaat niet altijd het correcte resultaat geven bij speciale formaten

Eenmaal de kolom van het type datetime is kan je heel gemakkelijk deze kolommen vergelijken en er zaken gaan uithalen zoals de maand waarin de wedstrijd plaatsvond. 

In [83]:
df_ginf.head()
df_ginf.info() # standaard ingelezen als string (dtype=object)
df_ginf.date = pd.to_datetime(df_ginf.date, format="%Y/%m/%d") # infer_datetime_format=False op True zetten indien formaat niet gekend, dit is wel trager
df_ginf.info()
df_ginf.head()

df_ginf.date.dt.month

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10112 entries, 0 to 10111
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id_odsp    10112 non-null  object        
 1   link_odsp  10112 non-null  object        
 2   adv_stats  10112 non-null  bool          
 3   date       10112 non-null  datetime64[ns]
 4   league     10112 non-null  object        
 5   season     10112 non-null  int64         
 6   country    10112 non-null  object        
 7   ht         10112 non-null  object        
 8   at         10112 non-null  object        
 9   fthg       10112 non-null  int64         
 10  ftag       10112 non-null  int64         
 11  odd_h      10112 non-null  float64       
 12  odd_d      10112 non-null  float64       
 13  odd_a      10112 non-null  float64       
 14  odd_over   977 non-null    float64       
 15  odd_under  977 non-null    float64       
 16  odd_bts    977 non-null    float64      

,id_odsp,link_odsp,adv_stats,date,league,season,country,ht,at,fthg,ftag,odd_h,odd_d,odd_a,odd_over,odd_under,odd_bts,odd_bts_n
0,UFot0hit/,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN
1,Aw5DflLH/,/soccer/germany/bundesliga-2011-2012/augsburg-...,True,2011-08-06,D1,2012,germany,FC Augsburg,SC Freiburg,2,2,2.36,3.60,3.40,NaN,NaN,NaN,NaN
2,bkjpaC6n/,/soccer/germany/bundesliga-2011-2012/werder-br...,True,2011-08-06,D1,2012,germany,Werder Bremen,Kaiserslautern,2,0,1.83,4.20,4.80,NaN,NaN,NaN,NaN
3,CzPV312a/,/soccer/france/ligue-1-2011-2012/paris-sg-lori...,True,2011-08-06,F1,2012,france,Paris Saint-Germain,Lorient,0,1,1.55,4.50,9.40,NaN,NaN,NaN,NaN
4,GUOdmtII/,/soccer/france/ligue-1-2011-2012/caen-valencie...,True,2011-08-06,F1,2012,france,Caen,Valenciennes,1,0,2.50,3.40,3.45,NaN,NaN,NaN,NaN


### Data shuffling

Een mogelijke manier om de privacy van je klanten te beschermen die we aangehaald hebben is data shuffling.
Bij deze methode is het de bedoeling dat je de namen in een dataset op een willekeurige manier door elkaar haalt.
Dit kan gedaan worden door gebruik te maken van de permutation methode van numpy  (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html) op de volgende manier:

In [79]:
display(df_events_copy_reduced.player.head())
test = df_events_copy_reduced.sample(frac=1) # fractie =1 is 100% van de samples terug maar in willekeurige volgorde
display(test.player.head())

0       mladen petric
1    dennis diekmeier
2    heiko westermann
3         sven bender
4         gokhan tore
Name: player, dtype: object

862823               aduriz
390237        jordan loties
881805        marco benassi
479794         andreas wolf
392272    sta©phane ruffier
Name: player, dtype: object

## Oefeningen

Om meer vertrouwd te worden met het werken met pandas kan je de volgende zaken zelf proberen.
Indien het niet lukt, mag je het me gerust vragen of het proberen zelf op te zoeken naar wat de oorzaak is.

Bewerkingen op de events.csv dataset:
* Controleer de kolommen "id_event", "id_odsp" en "sort_order". Is er 1 van deze drie overbodig? Verwijder de kolom indien dit zo is.
* Hernoem "Hamburg SV" naar "Hamburg FC". Let op, doe dit voor alle kolommen waar het kan staan.

Bewerkingen op de ginf.csv dataset:
* Maak One Hot Encoding voor de kolom "Country"
* Voeg een kolom toe dat drie waarden kan hebben: Home team wins, Home team loses, draw
* Maak nu ook een One Hot Encoding voor deze nieuwe kolom
* Drop de kolom met de urls voor de ploegen
* Bewaar je aangepaste dataset in "wedstrijden.csv" (Gebruik de functie to_save: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html)